In [1]:
import requests
import json
from typing import Any
from collections import defaultdict

TOKEN_DECIMALS: int = 9
first_n = 10

url = "https://api.subquery.network/sq/CommuneExplorer/commune-indexer"

def get_query(address: str) -> dict[Any, Any]:
    return f"""
    query{{
      transfers(
        filter: {{
          or:  [ 
          {{
            to: {{ equalTo: "{address}" }}
          }}
          ]
        }}
        orderBy: AMOUNT_DESC
      ) {{
        nodes {{
          to
          from
          blockNumber
          amount
        }}
      }}
    }}
    """


def aggregate_rewards(dict: [Any, Any]):
    pass
    
def get_transfer_data(wallet: str) -> dict[Any, Any] | None:
    query = get_query(wallet)
    payload = {"query": query}
    headers = {"Content-Type": "application/json"}
    response = requests.post(url, json=payload, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Request failed with status code: {response.status_code}")
        exit(1)

# This aggregates complete transfer analysis, unique wallets, with aggregated sorted transfer amounts.
def process_transfer_data(data: Any, wallet: str) -> dict[str, float]:
    output: dict[str, float] = {}
    wallet_totals = defaultdict(int)
    for transfer in data['data']['transfers']['nodes']:
        if transfer["to"] == wallet:
            wallet_totals[transfer['from']] += int(transfer['amount'])

    sorted_wallets = sorted(wallet_totals.items(), key=lambda x: x[1], reverse=True)

    for wallet, total in sorted_wallets:
        output[wallet] = total / 10**TOKEN_DECIMALS
   
    return output
        
# Figure out what accounts are transfering the most to exchanges        
def get_mexc_data() -> dict[str, float]:
    # this can be repeated for other exchanges in the future
    mexc_hotwallet: str = "5Ey2oJtrRiRYceYdESf647xKJzKJuWjnpuBSegnaWoSAJnDN"
    data = get_transfer_data(mexc_hotwallet)
    return process_transfer_data(data, mexc_hotwallet)


print("individual accounts, and their dump / transfer stack")
mexc_data = (get_mexc_data())
for k, v in list(mexc_data.items())[:first_n]:
    print(f"{k}: {v}")

individual accounts, and their dump / transfer stack
5GekCtvcjBA5RssJGVFptQZ99w6j4Am3TuTkW78p38zB8Umz: 746092.047142629
5CwDuQN67okvsPiFv46MauwPY5igbCfRjzeCxyKdgd8aoagF: 288025.054113565
5Gb7R56Ja6VVxVC91rGqzsfchucxqakKyB8pPxs5ERnuLzSG: 237646.401393863
5FKZA9aqRuvYiZVVh2WTrtjVYmWAre9Rw462FWLkc8VhCcka: 225781.254083815
5C7aZ2JWC4QneBTEppovyMMuNooRwkwQ118PsMcPcqvAbFTE: 199999.701394364
5Cwg8ne3DJsmnhLAWqJ4nj4CQEyTN2se6bdKsEw5VKqyQV3P: 149999.710822714
5GQncmGPze7UFqb2eTchxPPes5quzyNuGA6iiPY1rhpfyQRg: 140543.349760541
5Dsy7pPwFuAtccQsBuzYr59uHbA9Fege6dFc4au6F49bTR3j: 138999.701394363
5GuyTDRUWMF9XwWaGhRXFTyg5SiqLxk6wP49CA4JcXMUS2Mr: 122676.076089691
5GQpe94sZPxcBATZKANkRvZb39cv4yqMdUNdCZtdGjwVUTfF: 121862.503290855


In [3]:
# Now figure out where the accounts are getting the tokens from, we will go over first n
import tabulate

for address, amount in list(mexc_data.items())[:first_n]:
    print(f"to {address}, with the amount of {amount}")
    print("==================")
    data = get_transfer_data(address)
    processed = (process_transfer_data(data, address))
    for k, v in processed.items():
        print(f"from {k}, with the amount of {v}")
    print("---------------")
    

ModuleNotFoundError: No module named 'tabulate'